# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,12.92,55,85,6.49,RU,1689583645
1,1,krasnokholmskiy,55.9873,55.0466,22.47,63,99,4.20,RU,1689583646
2,2,port-aux-francais,-49.3500,70.2167,1.55,95,100,15.10,TF,1689583646
3,3,vorgashor,67.5833,63.9500,16.57,81,100,6.07,RU,1689583646
4,4,talnakh,69.4865,88.3972,10.67,86,100,5.00,RU,1689583646


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_filtered_df = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)].copy()
city_filtered_df = city_filtered_df [(city_filtered_df['Wind Speed'] < 4.5) & (city_filtered_df['Cloudiness'] == 0) ]
# Drop any rows with null values
city_filtered_df.dropna(inplace=True)

# Display sample data
print(len(city_filtered_df))
city_filtered_df.head()


18


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,fort bragg,35.1390,-79.0060,23.44,93,0,1.54,US,1689583647
69,69,alushta,44.6667,34.4167,26.63,46,0,0.81,UA,1689583659
130,130,richards bay,-28.7830,32.0377,22.08,55,0,3.49,ZA,1689583671
156,156,filadelfia,39.9523,-75.1638,23.95,94,0,3.09,US,1689583502
172,172,hamilton,39.1834,-84.5333,21.69,92,0,4.12,US,1689583529


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,fort bragg,US,35.1390,-79.0060,93,
69,alushta,UA,44.6667,34.4167,46,
130,richards bay,ZA,-28.7830,32.0377,55,
156,filadelfia,US,39.9523,-75.1638,94,
172,hamilton,US,39.1834,-84.5333,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
alushta - nearest hotel: Riviera Sunrise Resort & Spa
richards bay - nearest hotel: Bon Hotel Waterfront
filadelfia - nearest hotel: The Ritz-Carlton, Philadelphia
hamilton - nearest hotel: North Vista Manor
anadyr - nearest hotel: Гостевой дом
mahina - nearest hotel: Motu Martin
east london - nearest hotel: No hotel found
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
espanola - nearest hotel: Santa Claran Hotel & Casino
luau - nearest hotel: No hotel found
helena-west helena - nearest hotel: Best Western
corsicana - nearest hotel: Comfort Inn
cushing - nearest hotel: Executive Inn & Suites
san ignacio - nearest hotel: Rainforest Haven Inn
shchekino - nearest hotel: Щёкино
madalena - nearest hotel: Hotel de Paredes
romorantin-lanthenay - nearest hotel: Le Cheval Blanc


,City,Country,Lat,Lng,Humidity,Hotel Name
10,fort bragg,US,35.1390,-79.0060,93,Airborne Inn Lodging
69,alushta,UA,44.6667,34.4167,46,Riviera Sunrise Resort & Spa
130,richards bay,ZA,-28.7830,32.0377,55,Bon Hotel Waterfront
156,filadelfia,US,39.9523,-75.1638,94,"The Ritz-Carlton, Philadelphia"
172,hamilton,US,39.1834,-84.5333,92,North Vista Manor
173,anadyr,RU,64.7500,177.4833,40,Гостевой дом
209,mahina,PF,-17.5065,-149.4890,73,Motu Martin
224,east london,ZA,-33.0153,27.9116,49,No hotel found
262,galliano,US,29.4422,-90.2992,89,Fairfield Inn & Suites by Marriott Cut Off-Gal...
284,espanola,US,35.9911,-106.0806,42,Santa Claran Hotel & Casino


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols = ['Hotel Name','Country']
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

### I'm so sorry, i've troubles with my github files 